In [12]:
import pandas as pd
import numpy as np
import xgboost as xgb
from numpy import genfromtxt

In [3]:
df = pd.read_csv("/workspaces/SigmoidFreud/Data/X_test_window_size_5_time_encoding_True.csv", sep=',')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10704 entries, 0 to 10703
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   total_hours             10704 non-null  float64
 1   dlvry_weekend           10704 non-null  float64
 2   dlvry_bank_holiday      10704 non-null  float64
 3   dlvry_day_sin           10704 non-null  float64
 4   dlvry_day_cos           10704 non-null  float64
 5   dlvry_weekday_sin       10704 non-null  float64
 6   dlvry_weekday_cos       10704 non-null  float64
 7   dlvry_hour_sin          10704 non-null  float64
 8   dlvry_hour_cos          10704 non-null  float64
 9   lasttrade_weekend       10704 non-null  float64
 10  lasttrade_bank_holiday  10704 non-null  float64
 11  lasttrade_day_sin       10704 non-null  float64
 12  lasttrade_day_cos       10704 non-null  float64
 13  lasttrade_weekday_sin   10704 non-null  float64
 14  lasttrade_weekday_cos   10704 non-null

In [5]:
df.head(5)

,total_hours,dlvry_weekend,dlvry_bank_holiday,dlvry_day_sin,dlvry_day_cos,dlvry_weekday_sin,dlvry_weekday_cos,dlvry_hour_sin,dlvry_hour_cos,lasttrade_weekend,...,14,15,16,17,18,19,20,21,22,23
0,16.999015,1.0,0.0,0.571268,0.820763,-0.974928,-0.222521,1.0,6.123234e-17,0.0,...,-7.28,7.99,57.0,840.0,-9.32,-3.03,1.02,-5.92,16.3,780.0
1,16.999015,1.0,0.0,0.571268,0.820763,-0.974928,-0.222521,1.0,6.123234e-17,0.0,...,1.02,-5.92,16.3,780.0,10.52,-0.61,10.52,2.28,-72.0,720.0
2,16.999015,1.0,0.0,0.571268,0.820763,-0.974928,-0.222521,1.0,6.123234e-17,0.0,...,10.52,2.28,-72.0,720.0,1.00,1.59,-6.48,-6.57,321.8,660.0
3,16.999015,1.0,0.0,0.571268,0.820763,-0.974928,-0.222521,1.0,6.123234e-17,0.0,...,-6.48,-6.57,321.8,660.0,-7.86,-8.54,-9.76,-4.86,428.7,600.0
4,16.999015,1.0,0.0,0.571268,0.820763,-0.974928,-0.222521,1.0,6.123234e-17,0.0,...,-9.76,-4.86,428.7,600.0,-8.15,-3.65,-1.05,-0.68,-511.0,540.0


In [6]:
df.shape

(10704, 41)

In [18]:
X_train = genfromtxt("/workspaces/SigmoidFreud/Data/X_train_window_size_5_time_encoding_True.csv", delimiter=',', skip_header=1)
X_valid = genfromtxt("/workspaces/SigmoidFreud/Data/X_valid_window_size_5_time_encoding_True.csv", delimiter=',', skip_header=1)
X_test = genfromtxt("/workspaces/SigmoidFreud/Data/X_test_window_size_5_time_encoding_True.csv", delimiter=',', skip_header=1)

y_train = genfromtxt("/workspaces/SigmoidFreud/Data/y_train_window_size_5_time_encoding_True.csv", delimiter=',', skip_header=1)
y_valid = genfromtxt("/workspaces/SigmoidFreud/Data/y_valid_window_size_5_time_encoding_True.csv", delimiter=',', skip_header=1)
y_test = genfromtxt("/workspaces/SigmoidFreud/Data/y_test_window_size_5_time_encoding_True.csv", delimiter=',', skip_header=1)


In [33]:
from sklearn.metrics import mean_squared_error 
from math import sqrt
def evaluate_model(model, X_valid, y_valid_true):
    predictions = model.predict(X_valid)
    rms = sqrt(mean_squared_error(y_valid_true, predictions))
    print("Root mean squared error on valid:",rms)
    return rms

In [40]:
dmatrix_train = xgb.DMatrix(data=X_train,label=y_train) 
dmatrix_valid = xgb.DMatrix(data=X_valid,label=y_valid)
dmatrix_test = xgb.DMatrix(data=X_test,label=y_test)

In [43]:
params = {'objective': 'reg:linear', 'eval_metric': 'rmse', 'n_estimators': 100000}
evallist = [(dmatrix_valid, 'eval'), (dmatrix_train, 'train')]
xg_reg = xgb.train(params = params, dtrain=dmatrix_train, num_boost_round=50, evals=evallist)

[17:09:39] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[17:09:39] WARNING: ../src/learner.cc:573: 
Parameters: { "n_estimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	eval-rmse:5.14403	train-rmse:3.39889
[1]	eval-rmse:5.17879	train-rmse:3.34742
[2]	eval-rmse:5.18712	train-rmse:3.31872
[3]	eval-rmse:5.20033	train-rmse:3.26728
[4]	eval-rmse:5.20291	train-rmse:3.25185
[5]	eval-rmse:5.18161	train-rmse:3.23683
[6]	eval-rmse:5.18082	train-rmse:3.20342
[7]	eval-rmse:5.17404	train-rmse:3.18230
[8]	eval-rmse:5.18516	train-rmse:3.16680
[9]	eval-rmse:5.19832	train-rmse:3.13192
[10]	eval-rmse:5.21399	train-rmse:3.11073
[11]	eval-rmse:5.21575	train-rmse:3.10428
[12]	eval-rmse:5.21807	train-rmse:3.08910
[13]	eval-r

In [44]:
result = evaluate_model(xg_reg, dmatrix_test, dmatrix_test.get_label())

Root mean squared error on valid: 4.8639959706365055


In [25]:
from sklearn.ensemble import RandomForestRegressor
RF_base_model = RandomForestRegressor(n_estimators = 10, random_state=42069, n_jobs= -1)
RF_base_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=42069)

In [31]:
result = evaluate_model(RF_base_model, X_test, y_test)

Root mean squared error on valid: 5.075530350490662
